# Import library

In [1]:
import os
import yaml

with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

---

# Data validation

## Normal python: without validation

In [2]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name  = name
        self.age   = age
        self.email = email

In [3]:
foo = User(name="Joe", age='XXX', email="joe@gmail.com")
foo, foo.age

(<__main__.User at 0xffff9856fc40>, 'XXX')

## Pydantic: with validation

In [4]:
from typing import List
from pydantic import BaseModel, Field

In [5]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [6]:
foo_p = pUser(name="Jane", age=32, email='jane@gmail.com')
foo_p

pUser(name='Jane', age=32, email='jane@gmail.com')

In [7]:
foo_p = pUser(name="Jane", age='32', email='jane@gmail.com')
foo_p

pUser(name='Jane', age=32, email='jane@gmail.com')

In [8]:
foo_p = pUser(name="Jane", age='X', email='jane@gmail.com')
foo_p

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [9]:
class Class(BaseModel):
    students: List[pUser]

In [10]:
obj = Class(students=[
    pUser(name='Jane', age=32, email='jane@gmail.com')
])
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# Pydantic to OpenAI function definition

Docstring 잘 지켜서 pydantic BaseModel을 상속하는 class를 잘 만들면 OpenAI function으로 자동으로 바꿔준다.

In [11]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [12]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


- JSON schema (covered in 1st lesson)

In [13]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [14]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [15]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [16]:
class WeatherSearch2(BaseModel):
    airport_code: str

In [17]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

# Combining OpenAI functions with LangChain Expression Language

In [18]:
from langchain.chat_models import ChatOpenAI

In [19]:
model = ChatOpenAI()
model.invoke("What is the weather in SF today?", functions=[weather_function])

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [20]:
model_with_function = model.bind(functions=[weather_function])

In [21]:
model_with_function.invoke("What is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [22]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={'name': "WeatherSearch"})

In [23]:
model_with_forced_function.invoke("hi?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LAX"\n}', 'name': 'WeatherSearch'}})

# Using in a chain

In [24]:
from langchain.prompts import ChatPromptTemplate

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant"),
    ('user', "{input}")
])
chain = prompt | model_with_function
chain.invoke({'input': "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

# Using multiple functions

In [26]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [27]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]

In [28]:
model_with_functions = model.bind(functions=functions)
model_with_functions.invoke("What is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [29]:
model_with_functions.invoke("What are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "artist_name": "taylor swift",\n  "n": 3\n}', 'name': 'ArtistSearch'}})

In [30]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')